In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/u/shuhan/projects/vla')

from src.environments.highway_env.dataset import HighwayDataset, collate_fn

In [13]:
import torch
from torch.utils.data import DataLoader

data_folder = '/storage/Datasets/highway_env/highway_fast_v0_dqn_meta_action/rollouts'
dataset = HighwayDataset(data_folder, overfit=False)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [14]:
for batch in dataloader:
    break

In [15]:
from src.models.vlas.cont_obs_token_action import ContObsTokenActionVLA
from transformers import AutoModelForCausalLM, AutoTokenizer

llm_backbone = AutoModelForCausalLM.from_pretrained('gpt2')

hidden_dim = 768
obs_dim = 25
num_actions = 5
mlp_layers = 2

model = ContObsTokenActionVLA(llm_backbone, obs_dim, num_actions, hidden_dim, mlp_layers)

In [16]:
import os
# ckpt = os.path.expanduser('~/results/vla/quick_run/obs_reconst_only/lightning_logs/version_0/checkpoints/epoch=49-step=15650.ckpt')
# ckpt = os.path.expanduser('~/results/vla/quick_run/action_obs_overfit_1_seq_from_obs_pt/lightning_logs/version_0/checkpoints/epoch=16-step=5321.ckpt')
ckpt = os.path.expanduser('~/results/vla/quick_run/action_only/lightning_logs/version_1/checkpoints/epoch=413-step=129582.ckpt')

lg_ckpt = torch.load(ckpt, map_location='cpu', weights_only=True)['state_dict']
ori_ckpt = {}
for k, v in lg_ckpt.items():
    if k.startswith('vla.'):
        ori_ckpt[k[4:]] = v

model.load_state_dict(ori_ckpt)

<All keys matched successfully>

In [17]:
for current_step in range(1, 10):
  observations = batch[0][0, :current_step]
  past_actions = batch[1][0, :current_step-1]
  pred_action = model.predict_action(observations, past_actions)

  print(pred_action.argmax(dim=-1))

tensor(3)
tensor(2)
tensor(1)
tensor(1)
tensor(1)
tensor(3)
tensor(2)
tensor(3)
tensor(1)


In [18]:
observations.shape

torch.Size([9, 5, 5])

In [19]:
batch[1][0, current_step]

tensor(4.)

In [20]:
pred_action

tensor([8.4258e-13, 1.0000e+00, 1.9532e-12, 4.5175e-09, 5.8473e-10])

In [21]:
outputs, predictions, loss_dict = model(*batch)

In [22]:
predictions['action'].argmax(dim=-1)[0]

tensor([3, 2, 1, 1, 1, 3, 2, 3, 1, 4, 4, 2, 2, 3, 1, 1, 3, 4, 0, 4, 4, 0, 0, 1,
        1, 1, 0, 1, 1, 4])

In [26]:
((predictions['action'].argmax(dim=-1) == batch[1])[batch[2]]).float().mean()

tensor(0.9987)

In [43]:
outputs.keys()

odict_keys(['logits', 'past_key_values', 'hidden_states'])